## **1. 기본 작업**

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchtext.legacy import data, datasets
import time, random

In [2]:
SEED = 1234
random.seed(SEED)
torch.manual_seed(SEED)

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

## **2. 훈련 데이터에 대한 이해**

### **1) 필드 정의하기**

이번에 사용할 데이터는 총 3개의 열, 즉 다시 말해 3개의 필드를 가지고 있습니다. 왜냐하면 레이블이 총 2개이기 때문인데 이 중 1개만 사용할 것이지만 원활하게 데이터를 불러오기 위해서 일단은 3개 필드 모두 정의합니다.

In [4]:
# 3개의 필드 정의
TEXT = data.Field(lower = True)
UD_TAGS = data.Field(unk_token=None)
PTB_TAGS = data.Field(unk_token=None)

fields = (("text", TEXT), ("udtags", UD_TAGS), ("ptbtags", PTB_TAGS))

### **2) 데이터셋 만들기**

이제 토치텍스트에서 제공하는 훈련 데이터를 불러오는 동시에 데이터셋을 만들어보겠습니다. 훈련 데이터, 검증 데이터, 테스트 데이터를 각각 나눠서 저장합니다.

In [5]:
train_data, valid_data, test_data = datasets.UDPOS.splits(fields)

훈련 데이터, 검증 데이터, 테스트 데이터의 크기를 확인해보겠습니다.

In [6]:
print(f"훈련 샘플의 개수 : {len(train_data)}")
print(f"검증 샘플의 개수 : {len(valid_data)}")
print(f"테스트 샘플의 개수 : {len(test_data)}")

훈련 샘플의 개수 : 12543
검증 샘플의 개수 : 2002
테스트 샘플의 개수 : 2077


데이터셋을 생성하였으니 훈련 데이터의 필드들을 출력해봅시다.

In [7]:
# 훈련 데이터의 3개의 필드 확인
print(train_data.fields)

{'text': <torchtext.legacy.data.field.Field object at 0x0000024991E8DCA0>, 'udtags': <torchtext.legacy.data.field.Field object at 0x0000024991E8D280>, 'ptbtags': <torchtext.legacy.data.field.Field object at 0x0000024991E8D9A0>}


훈련 데이터의 첫번째 샘플에서 text와 두 개의 레이블을 모두 출력해보겠습니다.

In [8]:
# 첫번째 훈련 샘플의 text 필드
print(vars(train_data.examples[0])['text'])

['al', '-', 'zaman', ':', 'american', 'forces', 'killed', 'shaikh', 'abdullah', 'al', '-', 'ani', ',', 'the', 'preacher', 'at', 'the', 'mosque', 'in', 'the', 'town', 'of', 'qaim', ',', 'near', 'the', 'syrian', 'border', '.']


In [9]:
# 첫번째 훈련 샘플의 udtags 필드
print(vars(train_data.examples[0])['udtags'])

['PROPN', 'PUNCT', 'PROPN', 'PUNCT', 'ADJ', 'NOUN', 'VERB', 'PROPN', 'PROPN', 'PROPN', 'PUNCT', 'PROPN', 'PUNCT', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'PROPN', 'PUNCT', 'ADP', 'DET', 'ADJ', 'NOUN', 'PUNCT']


In [10]:
# 첫번째 훈련 샘플의 ptbdtags 필드
print(vars(train_data.examples[0])['ptbtags'])

['NNP', 'HYPH', 'NNP', ':', 'JJ', 'NNS', 'VBD', 'NNP', 'NNP', 'NNP', 'HYPH', 'NNP', ',', 'DT', 'NN', 'IN', 'DT', 'NN', 'IN', 'DT', 'NN', 'IN', 'NNP', ',', 'IN', 'DT', 'JJ', 'NN', '.']


### **3) Vocabulary 만들기**

이제 단어 집합을 생성해보겠습니다. 그리고 단어 집합을 생성 시에 사전 훈련된 워드 임베딩인 GloVe를 사용합니다.

In [11]:
# 최소 허용 빈도
MIN_FREQ = 5

# 사전 훈련된 워드 임베딩 GloVe 다운로드
TEXT.build_vocab(train_data, min_freq=MIN_FREQ, vectors="glove.6B.100d")
UD_TAGS.build_vocab(train_data)
PTB_TAGS.build_vocab(train_data)

상위 빈도 수 20개의 단어만 출력해보겠습니다. TEXT.vocab.freqs.most_common(20)을 통해 출력할 수 있습니다.

In [12]:
# 상위 빈도수 20개 단어
print(TEXT.vocab.freqs.most_common(20))

[('the', 9076), ('.', 8640), (',', 7021), ('to', 5137), ('and', 5002), ('a', 3782), ('of', 3622), ('i', 3379), ('in', 3112), ('is', 2239), ('you', 2156), ('that', 2036), ('it', 1850), ('for', 1842), ('-', 1426), ('have', 1359), ('"', 1296), ('on', 1273), ('was', 1244), ('with', 1216)]


영어는 보통 the가 빈도수가 가장 높습니다. 토치텍스트는 기본적으로 빈도수가 가장 높은 단어부터 작은 숫자를 부여합니다. 물론 < unk>는 0번, < pad>는 1번으로 자동 부여됩니다.

상위 정수 인덱스를 가진 10개의 단어를 출력합니다.

In [13]:
# 상위 정수 인덱스 단어 10개 출력
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', '.', ',', 'to', 'and', 'a', 'of', 'i']


이제 레이블의 단어 집합에 대해서 빈도수가 가장 높은 단어들과 그 빈도수를 출력해보겠습니다.

In [14]:
# 상위 빈도순으로 udtags 출력
print(UD_TAGS.vocab.freqs.most_common())

[('NOUN', 34781), ('PUNCT', 23679), ('VERB', 23081), ('PRON', 18577), ('ADP', 17638), ('DET', 16285), ('PROPN', 12946), ('ADJ', 12477), ('AUX', 12343), ('ADV', 10548), ('CCONJ', 6707), ('PART', 5567), ('NUM', 3999), ('SCONJ', 3843), ('X', 847), ('INTJ', 688), ('SYM', 599)]


In [15]:
# 상위 정수 인덱스 순으로 출력
print(UD_TAGS.vocab.itos)

['<pad>', 'NOUN', 'PUNCT', 'VERB', 'PRON', 'ADP', 'DET', 'PROPN', 'ADJ', 'AUX', 'ADV', 'CCONJ', 'PART', 'NUM', 'SCONJ', 'X', 'INTJ', 'SYM']


레이블에 속한 단어들의 분포를 출력해보겠습니다.

In [16]:
def tag_percentage(tag_conuts): # 태그 레이블의 분포를 확인하는 함수
    total_count = sum([count for tag, count in tag_conuts])
    tag_counts_percentage = [(tag, count, count/total_count) for tag, count in tag_conuts]
    return tag_counts_percentage

In [17]:
print("Tag  Occurences Percentage\n")
for tag, count, percent in tag_percentage(UD_TAGS.vocab.freqs.most_common()):
    print(f"{tag}\t{count}\t{percent*100:4.1f}%")

Tag  Occurences Percentage

NOUN	34781	17.0%
PUNCT	23679	11.6%
VERB	23081	11.3%
PRON	18577	 9.1%
ADP	17638	 8.6%
DET	16285	 8.0%
PROPN	12946	 6.3%
ADJ	12477	 6.1%
AUX	12343	 6.0%
ADV	10548	 5.2%
CCONJ	6707	 3.3%
PART	5567	 2.7%
NUM	3999	 2.0%
SCONJ	3843	 1.9%
X	847	 0.4%
INTJ	688	 0.3%
SYM	599	 0.3%


### **4) 데이터로더 만들기**

이제 데이터로더를 만들겠습니다. 배치 크기는 64로 합니다.

In [18]:
BATCH_SIZE = 64

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    device = device
)

첫번째 미니 배치만 꺼내서 미니 배치의 구성, 크기, text를 출력해보겠습니다.

In [19]:
batch = next(iter(train_iterator))
batch


[torchtext.legacy.data.batch.Batch of size 64 from UDPOS]
	[.text]:[torch.LongTensor of size 46x64]
	[.udtags]:[torch.LongTensor of size 46x64]
	[.ptbtags]:[torch.LongTensor of size 46x64]

In [20]:
batch.text.shape # (시퀀스 길이 x 배치 크기), batch_first=Ture를 안 했기에 배치 크기가 두번째 차원이 됩니다.

torch.Size([46, 64])

In [21]:
batch.text

tensor([[ 732,  167,    2,  ...,    2,   59,  668],
        [  16,  196,  133,  ..., 2991,   46,    1],
        [   1,   29,   48,  ..., 1582,   12,    1],
        ...,
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1]])

## **3. 모델 구현하기**

이제 모델을 구현하겠습니다. 기본적으로 대다대 RNN을 사용할텐데, 일단 양방향 여부와 층의 개수는 변수로 두겠습니다.

In [22]:
# 이번 모델에서는 batch_first=True를 사용하지 않으므로 배치 차원이 맨 앞이 아님
class RNNPOSTagger(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers, bidirectional=bidirectional)
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, text):
        # text = [sent len, batch size]
        embedd = self.dropout(self.embedding(text))
        
        # embedded = [sent len, batch size, emb dim]
        outputs, (hidden, cell) = self.rnn(embedd)
        
        # output = [sent len, batch size, hid dim * n directions]
        # hidden / cell = [n layers * n directions, batch size, hid dim]
        predictions = self.fc(self.dropout(outputs))
        
        # predictions = [sent len, batch size, output dim]
        return predictions

실제 클래스로부터 모델 객체로 생성 시, 양방향 여부를 True로 주고, 층의 개수를 2로 합니다.

In [23]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 128
OUTPUT_DIM = len(UD_TAGS.vocab)
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.25

model = RNNPOSTagger(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, N_LAYERS, BIDIRECTIONAL, DROPOUT)

In [24]:
# 파라미터 개수 출력
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"The model has {count_parameters(model):,} trainable parameters")

The model has 1,027,510 trainable parameters


총 102만 7천 5백 10개의 파라미터가 있습니다.

## **4. 사전 훈련된 워드 임베딩 사용하기**

앞서 언급했듯이, 이번 챕터에서는 사전 훈련된 워드 임베딩인 GloVe를 사용합니다. 이를 위해서 토치텍스트의 단어 집합 생성 시에 저장해두었던 GloVe 임베딩을 nn.Embedding()에 연결해줄 필요가 있습니다. 우선 단어 집합의 단어들에 맵핑된 사전 훈련된 워드 임베딩을 출력합니다.

In [25]:
pretrained_embeddings = TEXT.vocab.vectors
print(pretrained_embeddings.shape)

torch.Size([3921, 100])


단어 집합에 존재하는 총 3,921개의 단어에 대해서 100차원의 벡터가 맵핑되어져 있습니다. 이제 nn.Embedding()에 이를 연결시켜줍니다.

In [26]:
model.embedding.weight.data.copy_(pretrained_embeddings) # 임베딩 벡터값 copy

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.1020,  0.7700,  0.1169,  ..., -0.1416, -0.1932, -0.4225],
        [-0.0263,  0.0179, -0.5016,  ..., -0.8688,  0.9409, -0.2882],
        [ 0.1519,  0.4712,  0.0895,  ..., -0.4702, -0.3127,  0.1078]])

우선 < unk> 토큰의 인덱스와 < pad> 토큰의 인덱스를 저장해줍니다. 물론 각각 0과 1 인덱스입니다.

In [27]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
print(UNK_IDX)
print(PAD_IDX)

0
1


그리고 임의로 0번과 1번 단어에는 0벡터를 만들어줍니다.

In [28]:
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM) # 0번 임베딩 벡터에 0값을 채운다.
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM) # 1번 임베딩 벡터에 1값을 채운다.
print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.1020,  0.7700,  0.1169,  ..., -0.1416, -0.1932, -0.4225],
        [-0.0263,  0.0179, -0.5016,  ..., -0.8688,  0.9409, -0.2882],
        [ 0.1519,  0.4712,  0.0895,  ..., -0.4702, -0.3127,  0.1078]])


PAD 토큰과 UNK 토큰의 임베딩 벡터값이 0인 것을 볼 수 있습니다. 사전 훈련된 워드 임베딩을 사용할 준비가 되었습니다.

## **5. 옵티마이저와 비용 함수 구현하기**

옵티마이저 설계 전에 레이블 데이터의 패딩 토큰의 인덱스도 확인해봅시다.

In [29]:
TAG_PAD_IDX = UD_TAGS.vocab.stoi[UD_TAGS.pad_token]
print(TAG_PAD_IDX)

0


0인 것을 확인할 수 있습니다. 이를 하는 이유는 아래 비용 함수를 선택할 때 인자로 주기 위함입니다. 이제 옵티마이저를 설정합니다. 여기서는 Adma을 택했습니다.

In [30]:
optimizer = optim.Adam(model.parameters())

비용 함수로 크로스엔트로피 함수를 선택합니다. 이때 레이블 데이터의 패딩 토큰은 비용 함수의 연산에 포함시키지도 않도록 레이블 데이터의 패딩 토큰을 무시하라고 기재해줍니다.

In [31]:
criterion = nn.CrossEntropyLoss(ignore_index=TAG_PAD_IDX)

In [32]:
model = model.to(device)
criterion = criterion.to(device)

아직 모델은 훈련되지 않은 상태이지만 모델이 입력값을 넣어 출력(예측값)의 크기를 확인해보겠습니다. 여기서 넣는 입력값은 앞에서 꺼내두었던 첫번째 배치입니다.

In [33]:
prediction = model(batch.text)

In [34]:
prediction.shape

torch.Size([46, 64, 18])

46 x 64 x 18은 각각 (첫번째 배치의 시퀀스 길이 x 배치 크기 x 레이블 단어장의 크기)에 해당됩니다. 주의할 점은 현재 batch_first를 해주지 않아 배치 크기가 맨 앞 차원이 아니라는 것입니다. 또한 46은 첫번째 배치의 시퀀스 길이일뿐, 다른 배치들은 시퀀스 길이가 다를 수 있습니다.

이제 예측값에 대해서 시퀀스 길이와 배치 길이를 모두 펼쳐주는 작업을 해보겠습니다.

In [35]:
prediction = prediction.view(-1, prediction.shape[-1])
prediction.shape

torch.Size([2944, 18])

크기가 (2994 x 18)이 됩니다. 이번에는 첫번째 배치의 레이블 데이터의 크기를 보겠습니다.

In [36]:
batch.udtags.shape

torch.Size([46, 64])

46 x 64는 (첫번째 배치의 시퀀스 길이 x 배치 크기)에 해당됩니다. 이를 펼쳐보겠습니다.

In [37]:
batch.udtags.view(-1).shape

torch.Size([2944])

2,944의 크기를 가지게 됩니다.

## **6. 훈련과 평가하기**

In [38]:
def categorical_accuracy(preds, y, tag_pad_idx):
    """미니 배치에 대한 정확도 출력"""
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    non_pad_elements = (y != tag_pad_idx).nonzero()
    correct = max_preds[non_pad_elements].squeeze(1).eq(y[non_pad_elements])
    return correct.sum() / torch.FloatTensor([y[non_pad_elements].shape[0]])

In [39]:
def train(model, iterator, optimizer, criterion, tag_pad_idx):
    
    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for batch in iterator:

        text = batch.text
        tags = batch.udtags

        optimizer.zero_grad()

        #text = [sent len, batch size]     
        predictions = model(text)

        #predictions = [sent len, batch size, output dim]
        #tags = [sent len, batch size]
        predictions = predictions.view(-1, predictions.shape[-1]) # #predictions = [sent len * batch size, output dim]
        tags = tags.view(-1) # tags = [sent len * batch_size]

        #predictions = [sent len * batch size, output dim]
        #tags = [sent len * batch size]
        loss = criterion(predictions, tags)

        acc = categorical_accuracy(predictions, tags, tag_pad_idx)

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [40]:
def evaluate(model, iterator, criterion, tag_pad_idx):
    
    epoch_loss = 0
    epoch_acc = 0

    model.eval()

    with torch.no_grad():

        for batch in iterator:

            text = batch.text
            tags = batch.udtags

            predictions = model(text)

            predictions = predictions.view(-1, predictions.shape[-1])
            tags = tags.view(-1)

            loss = criterion(predictions, tags)

            acc = categorical_accuracy(predictions, tags, tag_pad_idx)

            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [41]:
N_EPOCHS = 10

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, TAG_PAD_IDX)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, TAG_PAD_IDX)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')

    print(f'Epoch: {epoch+1:02}')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01
	Train Loss: 1.090 | Train Acc: 66.15%
	 Val. Loss: 0.661 |  Val. Acc: 79.39%
Epoch: 02
	Train Loss: 0.387 | Train Acc: 87.81%
	 Val. Loss: 0.535 |  Val. Acc: 83.34%
Epoch: 03
	Train Loss: 0.303 | Train Acc: 90.21%
	 Val. Loss: 0.491 |  Val. Acc: 84.38%
Epoch: 04
	Train Loss: 0.264 | Train Acc: 91.41%
	 Val. Loss: 0.460 |  Val. Acc: 85.26%
Epoch: 05
	Train Loss: 0.237 | Train Acc: 92.24%
	 Val. Loss: 0.448 |  Val. Acc: 85.13%
Epoch: 06
	Train Loss: 0.219 | Train Acc: 92.75%
	 Val. Loss: 0.433 |  Val. Acc: 85.97%
Epoch: 07
	Train Loss: 0.204 | Train Acc: 93.33%
	 Val. Loss: 0.421 |  Val. Acc: 86.29%
Epoch: 08
	Train Loss: 0.191 | Train Acc: 93.72%
	 Val. Loss: 0.413 |  Val. Acc: 86.37%
Epoch: 09
	Train Loss: 0.178 | Train Acc: 94.04%
	 Val. Loss: 0.409 |  Val. Acc: 86.47%
Epoch: 10
	Train Loss: 0.169 | Train Acc: 94.36%
	 Val. Loss: 0.410 |  Val. Acc: 86.60%


In [42]:
test_loss, test_acc = evaluate(model, test_iterator, criterion, TAG_PAD_IDX)

print(f'Test Loss: {test_loss:.3f} |  Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.412 |  Test Acc: 87.26%
